<a href="https://colab.research.google.com/github/sangeetsaurabh/tweet_sentiment_extraction/blob/master/Data_preprocessing_for_biLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspellchecker

In [2]:
!pip install tokenizers

In [3]:
import pandas as pd
import numpy as np
import tokenizers
from spellchecker import SpellChecker
spell = SpellChecker()

import spacy
nlp = spacy.load('en')

pd.set_option("display.max_colwidth", None)



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data_folder = "/content/drive/My Drive/tweet-sentiment-extraction/data"
tmp_folder = '/tmp'

In [6]:
#### Copy the dataset
train_data = pd.read_csv(data_folder + "/train.csv")
test_data = pd.read_csv(data_folder + "/test.csv")

### drop null values row
train_data = train_data[train_data.text == train_data.text]

### Copy the original text into old_text. Just so we could compare later.
train_data["idx"] = train_data.index
test_data["idx"] = test_data.index

train_data["old_text"] = train_data.text
train_data["old_sel_text"] = train_data.selected_text

test_data["old_text"] = test_data.text



#### Make sure that selected text doesn't have the partial word

In [7]:
train_data.head()

,textID,text,selected_text,sentiment,idx,old_text,old_sel_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,0,"I`d have responded, if I were going","I`d have responded, if I were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,1,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD
2,088c60f138,my boss is bullying me...,bullying me,negative,2,my boss is bullying me...,bullying me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,3,what interview! leave me alone,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on the releases we already bought","Sons of ****,",negative,4,"Sons of ****, why couldn`t they put them on the releases we already bought","Sons of ****,"


##### Remove the special characters in the text

In [8]:
#### Update some of the texts manually. Found that some partial characters were causing issues.
#### There were more than 1900 such issues. Most of them were fixed prgrmatically below. Some of them are being fixed manually.

train_data.loc[707,'selected_text'] = ['He gives Twitter tips. Hope this helps']

train_data.loc[8729,'selected_text'] = ['lol']

train_data.loc[14611,'selected_text'] = ['so awesome']

train_data.loc[16100,'selected_text'] = ['that`s incredibly sweeeet!']

train_data.loc[16186,'selected_text'] = ['beautiful,']

train_data.loc[17762,'selected_text'] = ['It`s depressing.']

train_data.loc[19011,'selected_text'] = ['my pleasure!!']

train_data.loc[22782,'selected_text'] = ['wasn`t a good']

train_data.loc[23924,'selected_text'] = ['stinks!']

train_data.loc[24194,'selected_text'] = ['tried to make it sound']

train_data.loc[25691,'selected_text'] = ['@_Cheshire_Cat_']

train_data.loc[25901,'selected_text'] = ['don`t paint a good picture']

train_data.loc[26331,'selected_text'] = ['greedy.']

train_data.loc[3176,'selected_text'] = ['i agree it`s very important,']


In [9]:
#### clean up following characters
pd.set_option("display.max_colwidth", None)

### impacting 1 sentence
train_data['text'] = train_data['text'].str.replace('\xa0',' ')
train_data['selected_text'] = train_data['selected_text'].str.replace('\xa0',' ')


### Impacing 2 sentences
train_data['text'] = train_data['text'].str.replace('Â',' ')
train_data['selected_text'] = train_data['selected_text'].str.replace('Â',' ')

### Impacing 5 sentences
train_data['text'] = train_data['text'].str.replace('``',' ')
train_data['selected_text'] = train_data['selected_text'].str.replace('``',' ')

### Impacing 1 sentence
train_data['text'] = train_data['text'].str.replace('´','`')
train_data['selected_text'] = train_data['selected_text'].str.replace('´','`')

### Impacing 1 sentence
train_data['text'] = train_data['text'].str.replace('\t','')
train_data['selected_text'] = train_data['selected_text'].str.replace('\t','')

### Impacing 155 sentences
train_data['text'] = train_data['text'].str.replace('ï¿½',' ')
train_data['selected_text'] = train_data['selected_text'].str.replace('ï¿½',' ')

###Impacting 1220 rows (only 5 of them are in selected_text) (removing all the URLs)
train_data['text'] = train_data['text'].str.replace(r'http\S+', 'URL', regex=True)
train_data['selected_text'] = train_data['selected_text'].str.replace(r'http\S+', 'URL', regex=True)

#### Impacting 2 sentects. It seems to be a bug in the data, text doesn't have those characters 
train_data['selected_text'] = train_data['selected_text'].str.replace(r'ï', '', regex=True)

#### Impacting one sentence (no impact on test data, so just ok to remove it)
####Impacting 8 sentences. Most of them are neutral. Generally if a sentence has this, include in selected text
#train_data['text'] = train_data['text'].str.replace('\(:\|', '')
#train_data['selected_text'] = train_data['selected_text'].str.replace('\(:\|', '')


####Impacting 8 sentences. Most of them are neutral. Generally if a sentence has this, include in selected text
#train_data['text'] = train_data['text'].str.replace(':\|', '')
#train_data['text'] = train_data['text'].str.replace(':-\|', '')

#train_data['selected_text'] = train_data['selected_text'].str.replace(':\|', '')
#train_data['selected_text'] = train_data['selected_text'].str.replace(':-\|', '')

 

#### Remove some of the special characters emojis (impacts 5 to 7 rows)
#train_data['text'] = train_data['text'].str.replace(':\*\*-\(', '')
#train_data['text'] = train_data['text'].str.replace(':\*\*\(', '')
#train_data['text'] = train_data['text'].str.replace(':\*', '')

#train_data['selected_text'] = train_data['selected_text'].str.replace(':\*\*-\(', '')
#train_data['selected_text'] = train_data['selected_text'].str.replace(':\*\*\(', '')
#train_data['selected_text'] = train_data['selected_text'].str.replace(':\*', '')

####Impacting 20 sentences. Most of them are neutral. Generally if a sentence has this, include in selected text
#train_data['text'] = train_data['text'].str.replace(':O', '')
#train_data['selected_text'] = train_data['selected_text'].str.replace(':O', '')




###Remove leading and trailing spaces
train_data['text'] = train_data['text'].str.strip()
train_data['selected_text'] = train_data['selected_text'].str.strip()

train_data['text'] = train_data['text'].str.replace(r'\s{2,}', ' ',regex=True)
train_data['selected_text'] = train_data['selected_text'].str.replace(r'\s{2,}', ' ',regex=True)





### Impacing 1 sentence
test_data['text'] = test_data['text'].str.replace('Â',' ')

### Impacing 1 sentence
test_data['text'] = test_data['text'].str.replace('``',' ')


### Impacing 1 sentence
test_data['text'] = test_data['text'].str.replace('\t',' ')

### Impacing 13 sentences
test_data['text'] = test_data['text'].str.replace('ï¿½',' ')

### Impacting 174 rows (removing all the URLs)
test_data['text'] = test_data['text'].str.replace(r'http\S+', 'URL', regex=True)

### Impacting 2 rows. Just include this in the output
#test_data['text'] = test_data['text'].str.replace(':\|', '')
#test_data['text'] = test_data['text'].str.replace(':-\|', '')


### Impacting 1 row. Just include this in the output
test_data['text'] = test_data['text'].str.replace('¡', '')

#### Remove some of the special characters emojis (impacts 1 row)
#test_data['text'] = test_data['text'].str.replace(':\*', '')

#### Remove some of the special characters emojis (impacts 3 rows)
#test_data['text'] = test_data['text'].str.replace(':O', '')


###Remove leading, trailing and duplicate spaces
test_data['text'] = test_data['text'].str.strip()
test_data['text'] = test_data['text'].str.replace(r'\s{2,}', ' ',regex=True)






#### Get the Train Data Ready

In [10]:
def update_partial_token(row):
  return_row = row["sel_text_token"].copy()
  if len(row["set_diff"]) > 0:
    for item in row["set_diff"]:
      actual_text = [s for s in row["text_token"] if item in s]
      if len(actual_text) == 1:
        sel_txt_indx = return_row.index(item)
        return_row[sel_txt_indx] = actual_text[0]
      else:
        return_row = list(filter((item).__ne__,return_row))
  return return_row

        
    


In [11]:
sub =  r"[\w]+|[\d.#$%&+-/@,!?;\^_={}\(\)*:<>`'\d]+"

train_data["text"] = train_data["text"].str.lower()
train_data["selected_text"] = train_data["selected_text"].str.lower()

train_data["text_token"]= train_data["text"].str.findall(sub)
train_data["sel_text_token"]= train_data["selected_text"].str.findall(sub)

train_data["set_diff"] = train_data["sel_text_token"].apply(set) - train_data["text_token"].apply(set)

train_data["final_sel_text_token"] = train_data.apply(update_partial_token,axis=1)

#### Correct some of the rows manually

####selected_text feels incomplete for these 2 rows
#train_data.drop(index=8729,inplace=True)

train_data.loc[11808,'final_sel_text_token'] = [["hate"]]
train_data.loc[12522,'final_sel_text_token'] = [["goodbye"]]
train_data.loc[14172,'final_sel_text_token'] = [['bunny']]
train_data.loc[24929,'final_sel_text_token'] = [['wow']]
#train_data.loc[25691,'final_sel_text_token'] = [['_cheshire_cat_']]


train_data.loc[751,'selected_text'] = 'sucks'
train_data.loc[751,'final_sel_text_token'] = [train_data[train_data.idx == 751]['selected_text'].str.findall(sub).values[0]]

train_data.loc[4325,'selected_text'] = 'hoping'
train_data.loc[4325,'final_sel_text_token'] = [train_data[train_data.idx == 4325]['selected_text'].str.findall(sub).values[0]]

train_data.loc[7769,'selected_text'] = 'mean'
train_data.loc[7769,'final_sel_text_token'] = [train_data[train_data.idx == 7769]['selected_text'].str.findall(sub).values[0]]


train_data.loc[11507,'selected_text'] = 'heat'
train_data.loc[11507,'final_sel_text_token'] = [train_data[train_data.idx == 11507]['selected_text'].str.findall(sub).values[0]]

train_data.loc[14347,'selected_text'] = 'fuzzy'
train_data.loc[14347,'final_sel_text_token'] = [train_data[train_data.idx == 14347]['selected_text'].str.findall(sub).values[0]]

train_data.loc[15158,'selected_text'] = 'crash'
train_data.loc[15158,'final_sel_text_token'] = [train_data[train_data.idx == 15158]['selected_text'].str.findall(sub).values[0]]

train_data.loc[19476,'selected_text'] = 'hilarious'
train_data.loc[19476,'final_sel_text_token'] = [train_data[train_data.idx == 19476]['selected_text'].str.findall(sub).values[0]]

train_data.loc[22510,'selected_text'] = 'killing'
train_data.loc[22510,'final_sel_text_token'] = [train_data[train_data.idx == 22510]['selected_text'].str.findall(sub).values[0]]

train_data.loc[24223,'selected_text'] = 'cool'
train_data.loc[24223,'final_sel_text_token'] = [train_data[train_data.idx == 24223]['selected_text'].str.findall(sub).values[0]]



#train_data.loc[3176,'selected_text'] = 'i agree it`s very important,'
#train_data.loc[3176,'final_sel_text_token'] = [train_data[train_data.idx == 3176]['selected_text'].str.findall(sub).values[0]]

train_data["final_set_diff"] = train_data["final_sel_text_token"].apply(set) - train_data["text_token"].apply(set)

train_data[["text","text_token","selected_text","sel_text_token","set_diff","final_set_diff","final_sel_text_token","idx"]].iloc[11807:11808]

,text,text_token,selected_text,sel_text_token,set_diff,final_set_diff,final_sel_text_token,idx
11808,"4 hours of sleep, a migraine, again? what is wrong with me? hate my life","[4, hours, of, sleep, ,, a, migraine, ,, again, ?, what, is, wrong, with, me, ?, hate, my, life]",hat,[hat],{hat},{},[hate],11808


In [12]:
train_data[train_data['final_set_diff'].apply(len) > 0]

,textID,text,selected_text,sentiment,idx,old_text,old_sel_text,text_token,sel_text_token,set_diff,final_sel_text_token,final_set_diff


In [13]:
train_data[train_data['set_diff'].apply(len) > 0].shape

(1971, 12)

In [14]:
train_data["sel_text_token"] = train_data["final_sel_text_token"].copy()
train_data = train_data[["textID","text","text_token","selected_text","sel_text_token","sentiment","old_text","old_sel_text","idx"]]
train_data.head()

,textID,text,text_token,selected_text,sel_text_token,sentiment,old_text,old_sel_text,idx
0,cb774db0d1,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]",neutral,"I`d have responded, if I were going","I`d have responded, if I were going",0
1,549e992a42,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !!!]",sooo sad,"[sooo, sad]",negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,1
2,088c60f138,my boss is bullying me...,"[my, boss, is, bullying, me, ...]",bullying me,"[bullying, me]",negative,my boss is bullying me...,bullying me,2
3,9642c003ef,what interview! leave me alone,"[what, interview, !, leave, me, alone]",leave me alone,"[leave, me, alone]",negative,what interview! leave me alone,leave me alone,3
4,358bd9e861,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, ****,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","sons of ****,","[sons, of, ****,]",negative,"Sons of ****, why couldn`t they put them on the releases we already bought","Sons of ****,",4


In [15]:
train_data.iloc[14170:14173]

,textID,text,text_token,selected_text,sel_text_token,sentiment,old_text,old_sel_text,idx
14171,2e4c791638,yay! i finally made sales at the gallery; really helps when the artists are there,"[yay, !, i, finally, made, sales, at, the, gallery, ;, really, helps, when, the, artists, are, there]",helps,[helps],positive,Yay! I finally made sales at the gallery; Really helps when the artists are there,helps,14171
14172,c155efab1b,hey honey bunny here big bunny hugs,"[hey, honey, bunny, here, big, bunny, hugs]",unny,[bunny],positive,Hey Honey Bunny here big bunny hugs,unny,14172
14173,454efa3d41,"iya, nyokap gue pernah berkata demikian :-| and it`s so hard to share what we have","[iya, ,, nyokap, gue, pernah, berkata, demikian, :-, and, it, `, s, so, hard, to, share, what, we, have]",nd it`s so hard,"[and, it, `, s, so, hard]",negative,"iya, nyokap gue pernah berkata demikian :-| and it`s so hard to share what we have",nd it`s so hard,14173


#### Get the test data ready

In [16]:
pd.set_option("display.max_rows",500)

sub =  r"[\w]+|[\d.#$%&+-/@,!?;\^_={}\(\)*:<>`'\d]+"
test_data["text"] = test_data["text"].str.lower()
test_data["text_token"]= test_data["text"].str.findall(sub)

test_data.tail()

,textID,text,sentiment,idx,old_text,text_token
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep but i try it",negative,3529,"its at 3 am, im very tired but i can`t sleep but i try it","[its, at, 3, am, ,, im, very, tired, but, i, can, `, t, sleep, but, i, try, it]"
3530,416863ce47,"all alone in this old house again. thanks for the net which keeps me alive and kicking! whoever invented the net, i wanna kiss your hair!",positive,3530,"All alone in this old house again. Thanks for the net which keeps me alive and kicking! Whoever invented the net, i wanna kiss your hair!","[all, alone, in, this, old, house, again, ., thanks, for, the, net, which, keeps, me, alive, and, kicking, !, whoever, invented, the, net, ,, i, wanna, kiss, your, hair, !]"
3531,6332da480c,i know what you mean. my little dog is sinking into depression... he wants to move someplace tropical,negative,3531,I know what you mean. My little dog is sinking into depression... he wants to move someplace tropical,"[i, know, what, you, mean, ., my, little, dog, is, sinking, into, depression, ..., he, wants, to, move, someplace, tropical]"
3532,df1baec676,_sutra what is your next youtube video gonna be about? i love your videos!,positive,3532,_sutra what is your next youtube video gonna be about? I love your videos!,"[_sutra, what, is, your, next, youtube, video, gonna, be, about, ?, i, love, your, videos, !]"
3533,469e15c5a8,url - omgssh ang cute ng bby.!,positive,3533,http://twitpic.com/4woj2 - omgssh ang cute ng bby.!,"[url, -, omgssh, ang, cute, ng, bby, .!]"


How many words that are there in test text that are not there in the train text?

In [17]:
test_train_diff = set(test_data["text_token"].sum()) - set(train_data["text_token"].sum())
len(test_train_diff)

1910

#### Let's lemmatize to ensure that we reduce the number of missing text

In [18]:
### Lemmatize train words
unique_train_words = list(set(train_data["text_token"].sum()))

unique_lemma_train_words = []
train_lemma_words = {}

separator = ', '
doc = nlp(separator.join(unique_train_words))

for token in doc:
  if token.text != token.lemma_:
    train_lemma_words[token.text] = token.lemma_
  unique_lemma_train_words.append(token.lemma_)



In [19]:
### Lemmatize test words

unique_test_words = list(set(test_data["text_token"].sum()))

unique_lemma_test_words = []
test_lemma_words = {}

separator = ', '
doc = nlp(separator.join(unique_test_words))

for token in doc:
  if token.text != token.lemma_:
    test_lemma_words[token.text] = token.lemma_
  unique_lemma_test_words.append(token.lemma_)


In [20]:
print (f'The number of training words lemmatized: {len(train_lemma_words)}')
print (f'The number of training words lemmatized: {len(test_lemma_words)}')

The number of training words lemmatized: 4217
The number of training words lemmatized: 1315


In [21]:
### Number of words still in test set that's not part of training set
test_train_diff = set(unique_lemma_test_words) - set(unique_lemma_train_words)
len(test_train_diff)

1675

There are 1669 more words left. Let's include lemmatized words into test and train token.

In [22]:
def update_lemmatized_token(row,token_column,lemma_word_dict):
  updated_words = {}
  try:
    for i, item in enumerate(row[token_column]):
      if item in lemma_word_dict.keys():
        row[token_column][i] = lemma_word_dict[item]
        updated_words[lemma_word_dict[item]] = item
  except:
    print(row)
    print("\n")
  return updated_words


In [23]:
train_data.iloc[2365:2366]

,textID,text,text_token,selected_text,sel_text_token,sentiment,old_text,old_sel_text,idx
2366,38b714ce6c,i know - but there are loads of others left,"[i, know, -, but, there, are, loads, of, others, left]",i know - but there are loads of others left,"[i, know, -, but, there, are, loads, of, others, left]",neutral,I know - but there are loads of others left,I know - but there are loads of others left,2366


In [24]:
train_data["text_lemmetized_words"] = train_data.apply(update_lemmatized_token,args=("text_token",train_lemma_words),axis=1)
train_data["sel_text_lemmetized_words"] = train_data.apply(update_lemmatized_token,args=("sel_text_token",train_lemma_words),axis=1)

In [25]:
train_data.iloc[2365:2366]

,textID,text,text_token,selected_text,sel_text_token,sentiment,old_text,old_sel_text,idx,text_lemmetized_words,sel_text_lemmetized_words
2366,38b714ce6c,i know - but there are loads of others left,"[-PRON-, know, -, but, there, be, load, of, other, leave]",i know - but there are loads of others left,"[-PRON-, know, -, but, there, be, load, of, other, leave]",neutral,I know - but there are loads of others left,I know - but there are loads of others left,2366,"{'-PRON-': 'i', 'be': 'are', 'load': 'loads', 'other': 'others', 'leave': 'left'}","{'-PRON-': 'i', 'be': 'are', 'load': 'loads', 'other': 'others', 'leave': 'left'}"


In [26]:
train_data.iloc[5352:5353]

,textID,text,text_token,selected_text,sel_text_token,sentiment,old_text,old_sel_text,idx,text_lemmetized_words,sel_text_lemmetized_words
5353,4ead488f3e,i waited too long to buy pink tickets! now we`re going to be in nose bleed!! how sad we`re taking binoculars...,"[-PRON-, wait, too, long, to, buy, pink, ticket, !, now, -PRON-, `, be, go, to, be, in, nose, bleed, !!, how, sad, -PRON-, `, be, taking, binocular, ...]",w sad,[sad],negative,i waited too long to buy pink tickets! now we`re going to be in nose bleed!! how sad we`re taking binoculars...,w sad,5353,"{'-PRON-': 'we', 'wait': 'waited', 'ticket': 'tickets', 'be': 're', 'go': 'going', 'binocular': 'binoculars'}",{}


In [27]:
test_data["text_lemmetized_words"] = test_data.apply(update_lemmatized_token,args=("text_token",test_lemma_words),axis=1)

In [28]:
test_data.head()

,textID,text,sentiment,idx,old_text,text_token,text_lemmetized_words
0,f87dea47db,last session of the day url,neutral,0,Last session of the day http://twitpic.com/67ezh,"[last, session, of, the, day, url]",{}
1,96d74cb729,shanghai is also really exciting (precisely -- skyscrapers galore). good tweeps in china: (sh) (bj).,positive,1,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,"[shanghai, be, also, really, exciting, (, precisely, --, skyscraper, galore, )., good, tweep, in, china, :, (, sh, ), (, bj, ).]","{'be': 'is', 'skyscraper': 'skyscrapers', 'tweep': 'tweeps'}"
2,eee518ae67,"recession hit veronique branquinho, she has to quit her company, such a shame!",negative,2,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!","[recession, hit, veronique, branquinho, ,, -PRON-, have, to, quit, -PRON-, company, ,, such, going to, shame, !]","{'-PRON-': 'her', 'have': 'has', 'going to': 'a'}"
3,01082688c6,happy bday!,positive,3,happy bday!,"[happy, bday, !]",{}
4,33987a8ee5,url - i like it!!,positive,4,http://twitpic.com/4w75p - I like it!!,"[url, -, -PRON-, like, -PRON-, !!]",{'-PRON-': 'it'}


There are still 1660 words in test dataset that are not there in train. Let's do spelling correction on those words to see if that helps.

In [29]:
#misspelled = spell.unknown(test_train_diff)

In [30]:
#spell_corrected_word = []
#corrected_word_dict = {}

#for item in misspelled:
#  corrected_word_dict[item] = spell.correction(item)
 # spell_corrected_word.append(spell.correction(item))

In [31]:
def update_misspelled_token(row,token_column,mispell_word_dict):
  updated_words = {}
  for i, item in enumerate(row[token_column]):
    if item in mispell_word_dict.keys():
      row[token_column][i] = mispell_word_dict[item]
      updated_words[mispell_word_dict[item]] = item
  return updated_words

In [32]:
#test_data["text_misspelled_words"] = test_data.apply(update_misspelled_token,args=("text_token",corrected_word_dict),axis=1)

In [33]:
test_data.tail()

,textID,text,sentiment,idx,old_text,text_token,text_lemmetized_words
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep but i try it",negative,3529,"its at 3 am, im very tired but i can`t sleep but i try it","[-PRON-, at, 3, a.m., ,, im, very, tired, but, -PRON-, can, `, t, sleep, but, -PRON-, try, -PRON-]","{'-PRON-': 'it', 'a.m.': 'am'}"
3530,416863ce47,"all alone in this old house again. thanks for the net which keeps me alive and kicking! whoever invented the net, i wanna kiss your hair!",positive,3530,"All alone in this old house again. Thanks for the net which keeps me alive and kicking! Whoever invented the net, i wanna kiss your hair!","[all, alone, in, this, old, house, again, ., thank, for, the, net, which, keep, -PRON-, alive, and, kicking, !, whoever, invent, the, net, ,, -PRON-, wanna, kiss, -PRON-, hair, !]","{'thank': 'thanks', 'keep': 'keeps', '-PRON-': 'your', 'invent': 'invented'}"
3531,6332da480c,i know what you mean. my little dog is sinking into depression... he wants to move someplace tropical,negative,3531,I know what you mean. My little dog is sinking into depression... he wants to move someplace tropical,"[-PRON-, know, what, -PRON-, mean, ., -PRON-, little, dog, be, sinking, into, depression, ..., -PRON-, want, to, move, someplace, tropical]","{'-PRON-': 'he', 'be': 'is', 'want': 'wants'}"
3532,df1baec676,_sutra what is your next youtube video gonna be about? i love your videos!,positive,3532,_sutra what is your next youtube video gonna be about? I love your videos!,"[_sutra, what, be, -PRON-, next, youtube, video, gonna, be, about, ?, -PRON-, love, -PRON-, videos, !]","{'be': 'is', '-PRON-': 'your'}"
3533,469e15c5a8,url - omgssh ang cute ng bby.!,positive,3533,http://twitpic.com/4woj2 - omgssh ang cute ng bby.!,"[url, -, omgssh, ang, cute, ng, bby, .!]",{}


#### Save the train and test_dataset

In [34]:
### Drop the neutral for now
#train_data = train_data[train_data.sentiment != 'neutral']
#test_data = test_data[test_data.sentiment != 'neutral']

print(train_data.shape)
print(test_data.shape)

(27480, 11)
(3534, 7)


In [35]:
test_data.head()

,textID,text,sentiment,idx,old_text,text_token,text_lemmetized_words
0,f87dea47db,last session of the day url,neutral,0,Last session of the day http://twitpic.com/67ezh,"[last, session, of, the, day, url]",{}
1,96d74cb729,shanghai is also really exciting (precisely -- skyscrapers galore). good tweeps in china: (sh) (bj).,positive,1,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,"[shanghai, be, also, really, exciting, (, precisely, --, skyscraper, galore, )., good, tweep, in, china, :, (, sh, ), (, bj, ).]","{'be': 'is', 'skyscraper': 'skyscrapers', 'tweep': 'tweeps'}"
2,eee518ae67,"recession hit veronique branquinho, she has to quit her company, such a shame!",negative,2,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!","[recession, hit, veronique, branquinho, ,, -PRON-, have, to, quit, -PRON-, company, ,, such, going to, shame, !]","{'-PRON-': 'her', 'have': 'has', 'going to': 'a'}"
3,01082688c6,happy bday!,positive,3,happy bday!,"[happy, bday, !]",{}
4,33987a8ee5,url - i like it!!,positive,4,http://twitpic.com/4w75p - I like it!!,"[url, -, -PRON-, like, -PRON-, !!]",{'-PRON-': 'it'}


In [36]:
train_data["text"] = train_data["text_token"].apply(lambda x: ' '.join(x))
train_data["selected_text"] = train_data["sel_text_token"].apply(lambda x: ' '.join(x))
test_data["text"] = test_data["text_token"].apply(lambda x: ' '.join(x))

In [37]:
train_data.iloc[2365:2366]

,textID,text,text_token,selected_text,sel_text_token,sentiment,old_text,old_sel_text,idx,text_lemmetized_words,sel_text_lemmetized_words
2366,38b714ce6c,-PRON- know - but there be load of other leave,"[-PRON-, know, -, but, there, be, load, of, other, leave]",-PRON- know - but there be load of other leave,"[-PRON-, know, -, but, there, be, load, of, other, leave]",neutral,I know - but there are loads of others left,I know - but there are loads of others left,2366,"{'-PRON-': 'i', 'be': 'are', 'load': 'loads', 'other': 'others', 'leave': 'left'}","{'-PRON-': 'i', 'be': 'are', 'load': 'loads', 'other': 'others', 'leave': 'left'}"


In [38]:
test_data.head()

,textID,text,sentiment,idx,old_text,text_token,text_lemmetized_words
0,f87dea47db,last session of the day url,neutral,0,Last session of the day http://twitpic.com/67ezh,"[last, session, of, the, day, url]",{}
1,96d74cb729,shanghai be also really exciting ( precisely -- skyscraper galore ). good tweep in china : ( sh ) ( bj ).,positive,1,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,"[shanghai, be, also, really, exciting, (, precisely, --, skyscraper, galore, )., good, tweep, in, china, :, (, sh, ), (, bj, ).]","{'be': 'is', 'skyscraper': 'skyscrapers', 'tweep': 'tweeps'}"
2,eee518ae67,"recession hit veronique branquinho , -PRON- have to quit -PRON- company , such going to shame !",negative,2,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!","[recession, hit, veronique, branquinho, ,, -PRON-, have, to, quit, -PRON-, company, ,, such, going to, shame, !]","{'-PRON-': 'her', 'have': 'has', 'going to': 'a'}"
3,01082688c6,happy bday !,positive,3,happy bday!,"[happy, bday, !]",{}
4,33987a8ee5,url - -PRON- like -PRON- !!,positive,4,http://twitpic.com/4w75p - I like it!!,"[url, -, -PRON-, like, -PRON-, !!]",{'-PRON-': 'it'}


Add sentiment to the Text such that LSTM knows about the sentiment

In [39]:
train_data["text"] = train_data["sentiment"] + " " + train_data["text"] + " "+ train_data["sentiment"]
test_data["text"] = test_data["sentiment"] + " " + test_data["text"] + " "+ test_data["sentiment"]
train_data.head()

,textID,text,text_token,selected_text,sel_text_token,sentiment,old_text,old_sel_text,idx,text_lemmetized_words,sel_text_lemmetized_words
0,cb774db0d1,"neutral -PRON- ` would have respond , if -PRON- be go neutral","[-PRON-, `, would, have, respond, ,, if, -PRON-, be, go]","-PRON- ` would have respond , if -PRON- be go","[-PRON-, `, would, have, respond, ,, if, -PRON-, be, go]",neutral,"I`d have responded, if I were going","I`d have responded, if I were going",0,"{'-PRON-': 'i', 'would': 'd', 'respond': 'responded', 'be': 'were', 'go': 'going'}","{'-PRON-': 'i', 'would': 'd', 'respond': 'responded', 'be': 'were', 'go': 'going'}"
1,549e992a42,negative sooo sad -PRON- will miss -PRON- here in san diego !!! negative,"[sooo, sad, -PRON-, will, miss, -PRON-, here, in, san, diego, !!!]",sooo sad,"[sooo, sad]",negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,1,{'-PRON-': 'you'},{}
2,088c60f138,negative -PRON- boss be bullying -PRON- ... negative,"[-PRON-, boss, be, bullying, -PRON-, ...]",bullying -PRON-,"[bullying, -PRON-]",negative,my boss is bullying me...,bullying me,2,"{'-PRON-': 'me', 'be': 'is'}",{'-PRON-': 'me'}
3,9642c003ef,negative what interview ! leave -PRON- alone negative,"[what, interview, !, leave, -PRON-, alone]",leave -PRON- alone,"[leave, -PRON-, alone]",negative,what interview! leave me alone,leave me alone,3,{'-PRON-': 'me'},{'-PRON-': 'me'}
4,358bd9e861,"negative son of ****, why couldn ` t -PRON- put -PRON- on the release -PRON- already buy negative","[son, of, ****,, why, couldn, `, t, -PRON-, put, -PRON-, on, the, release, -PRON-, already, buy]","son of ****,","[son, of, ****,]",negative,"Sons of ****, why couldn`t they put them on the releases we already bought","Sons of ****,",4,"{'son': 'sons', '-PRON-': 'we', 'release': 'releases', 'buy': 'bought'}",{'son': 'sons'}


In [40]:
test_data.head()

,textID,text,sentiment,idx,old_text,text_token,text_lemmetized_words
0,f87dea47db,neutral last session of the day url neutral,neutral,0,Last session of the day http://twitpic.com/67ezh,"[last, session, of, the, day, url]",{}
1,96d74cb729,positive shanghai be also really exciting ( precisely -- skyscraper galore ). good tweep in china : ( sh ) ( bj ). positive,positive,1,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,"[shanghai, be, also, really, exciting, (, precisely, --, skyscraper, galore, )., good, tweep, in, china, :, (, sh, ), (, bj, ).]","{'be': 'is', 'skyscraper': 'skyscrapers', 'tweep': 'tweeps'}"
2,eee518ae67,"negative recession hit veronique branquinho , -PRON- have to quit -PRON- company , such going to shame ! negative",negative,2,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!","[recession, hit, veronique, branquinho, ,, -PRON-, have, to, quit, -PRON-, company, ,, such, going to, shame, !]","{'-PRON-': 'her', 'have': 'has', 'going to': 'a'}"
3,01082688c6,positive happy bday ! positive,positive,3,happy bday!,"[happy, bday, !]",{}
4,33987a8ee5,positive url - -PRON- like -PRON- !! positive,positive,4,http://twitpic.com/4w75p - I like it!!,"[url, -, -PRON-, like, -PRON-, !!]",{'-PRON-': 'it'}


In [41]:
valid_data = train_data.sample(frac=0.05)
train_data = train_data.loc[~train_data.index.isin(valid_data.index)]
print(valid_data.shape)
print(train_data.shape)

(1374, 11)
(26106, 11)


In [42]:
test_data["selected_text"] = test_data["old_text"].copy()
test_data.head()

,textID,text,sentiment,idx,old_text,text_token,text_lemmetized_words,selected_text
0,f87dea47db,neutral last session of the day url neutral,neutral,0,Last session of the day http://twitpic.com/67ezh,"[last, session, of, the, day, url]",{},Last session of the day http://twitpic.com/67ezh
1,96d74cb729,positive shanghai be also really exciting ( precisely -- skyscraper galore ). good tweep in china : ( sh ) ( bj ). positive,positive,1,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,"[shanghai, be, also, really, exciting, (, precisely, --, skyscraper, galore, )., good, tweep, in, china, :, (, sh, ), (, bj, ).]","{'be': 'is', 'skyscraper': 'skyscrapers', 'tweep': 'tweeps'}",Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).
2,eee518ae67,"negative recession hit veronique branquinho , -PRON- have to quit -PRON- company , such going to shame ! negative",negative,2,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!","[recession, hit, veronique, branquinho, ,, -PRON-, have, to, quit, -PRON-, company, ,, such, going to, shame, !]","{'-PRON-': 'her', 'have': 'has', 'going to': 'a'}","Recession hit Veronique Branquinho, she has to quit her company, such a shame!"
3,01082688c6,positive happy bday ! positive,positive,3,happy bday!,"[happy, bday, !]",{},happy bday!
4,33987a8ee5,positive url - -PRON- like -PRON- !! positive,positive,4,http://twitpic.com/4w75p - I like it!!,"[url, -, -PRON-, like, -PRON-, !!]",{'-PRON-': 'it'},http://twitpic.com/4w75p - I like it!!


In [43]:
train_data[["text","selected_text","idx","sentiment"]].to_csv(data_folder + "/train_transform.csv",index=False)
valid_data[["text","selected_text","idx","sentiment"]].to_csv(data_folder + "/valid_transform.csv",index=False)
test_data[["text","selected_text","idx","sentiment"]].to_csv(data_folder + "/test_transform.csv",index=False)

In [44]:
train_data[["text","selected_text","idx","sentiment"]].iloc[2365:2366]

,text,selected_text,idx,sentiment
2489,"positive how fun !! -PRON- like the beach , but -PRON- tend to get really dark fast . positive","!! -PRON- like the beach ,",2489,positive
